# **Seminar 2 - Attention и Transformer**
*Naumov Anton (Any0019)*

*To contact me in telegram: @any0019*

## 1. Построим Transformer с нуля в Pytorch

<img src="https://i.pinimg.com/736x/c1/83/1a/c1831a58ecc935fc0f2ef4d35ce4fddb.jpg">

In [ ]:
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
%matplotlib inline

### 1.1 Multi-head Attention

#### Основной класс - MultiHeadAttention

**Инициализация:**
* _in_size_ ~ размер эмбеддингов на входе
* _head_size_ ~ размер эмбеддингов матриц Q, K, V после преобразования
* _num_heads_ ~ число голов
* _out_size_ ~ размер эмбеддингов на выходе
* _query_in_size_ ~ размер эмбеддингов на входе для query (если None, то in_size)

**Forward:**
* _query, key, value_ ~ 3 тензора (по одному под Q, K и V преобразования - это ещё не сами тензоры $\text{batch_size} \times seq \times d_k$, а тензоры $\text{batch_size} \times seq \times \text{in_size}$)
* _mask_ ~ булева маска для Masked Multi-head Attention (в декодере)

$$ Attention(Q, K, V) = softmax\Bigg(\frac{QK^T}{\sqrt{d_k}}\Bigg) \cdot V $$
$$ MultiHead(Q, K, V) = Concat(head_1, ..., head_H) \cdot W^O \quad ; \quad head_i = Attention(Q W_i^Q, K W_i^K, V W_i^V)$$

In [ ]:
class MultiHeadAttention(nn.Module):
    """
    Class to calculate Multi-head attention (or Masked Multi-head attention for decoder) operation
    """
    def __init__(self, in_size, head_size, num_heads, out_size, query_in_size=None):
        """
        Args:
            in_size: embedding size of input
            head_size: hidden size of Q, K, V matrices
            num_heads: number of heads
            out_size: output embedding size
            query_in_size: embedding size of input for query (if not provided - same as in_size)
        """
        super(MultiHeadAttention, self).__init__()

        # Запишем все переданые гиперпараметры слоя
        self.in_size = in_size
        self.head_size = head_size
        self.num_heads = num_heads
        self.out_size = out_size
        self.query_in_size = self.in_size if query_in_size is None else query_in_size
       
        # Линейные преобразования для Q, K, V матриц (сразу хотим получить все Q, K, V матрицы)
        self.query_matrix = ...
        self.key_matrix = ...
        self.value_matrix = ...
        
        # Линейное преобразование для получения выхода после конкатенации голов
        self.out = ...

    def forward(self, query, key, value, mask=None):
        """
        Args:
           query : tensor for query
           key : tensor for key
           value : tensor for value
           mask: mask for decoder
        
        Returns:
           output vector from multihead attention
        """
        # Тензоры приходят размера batch_size x seq_len x in_size
        batch_size = key.size(0)
        seq_len = key.size(1)
        
        # Число токенов в query будет другим для decoder-а
        query_seq_len = query.size(1)
       
        # Применяем линейные преобразования на входе
        q = ...
        k = ...
        v = ...
       
        # Считаем релевантность
        relevance = ...
        
        # Если есть маска (для декодера), то заполняем значения по маске как минус бесконечность (чтобы exp(r) = 0 в softmax)
        if mask is not None:
             relevance = ...

        # Получаем вероятности
        relevance = ...
 
        # Считаем выходы из каждой головы
        head_i = ...
        
        # Конкатенируем выходы
        concat = ...
        
        # Финальное линейное преобразование
        return ...

#### Протестируем MultiHeadAttention для энкодера

In [ ]:
tmp_layer = MultiHeadAttention(
    in_size=10,
    head_size=4,
    num_heads=3,
    out_size=15,
)

tmp_layer

In [ ]:
# Проверяем в обычном прямом проходе из энкодера
tmp_input = torch.rand(2, 5, 10)

print("Encoder-like input, no mask")
print(f'Input shape: {tmp_input.shape}')
tmp_output = tmp_layer(tmp_input, tmp_input, tmp_input)
print(f'Output shape: {tmp_output.shape}')

del tmp_input, tmp_output

#### Протестируем MultiHeadAttention для смеси энкодера и декодера

In [ ]:
tmp_layer = MultiHeadAttention(
    in_size=10,
    head_size=4,
    num_heads=3,
    out_size=15,
    query_in_size=12,
)

tmp_layer

In [ ]:
# Проверяем в прямом проходе из декодера, где мы смешиваем информацию из энкодера и декодера
tmp_input_q = torch.rand(2, 5, 12)
tmp_input_kv = torch.rand(2, 7, 10)

print("Encoder+Decoder-like input, no mask")
print(f'Input Q shape: {tmp_input_q.shape}')
print(f'Input KV shape: {tmp_input_kv.shape}')

tmp_output = tmp_layer(tmp_input_q, tmp_input_kv, tmp_input_kv)
print(f'Output shape: {tmp_output.shape}')

del tmp_input_q, tmp_input_kv, tmp_output

#### Треугольная маска в декодере

In [ ]:
def make_decoder_mask(decoder_embed):
    """
    Make mask for decoder Masked Multi-head Attention based on input sequence
    Args:
        decoder_embed: decoder sequence after embed
    Returns:
        mask: mask for Masked Multi-head Attention
    """
    batch_size, decoder_seq_len, _ = decoder_embed.shape
    mask = torch.tril(torch.ones((decoder_seq_len, decoder_seq_len))).expand(
        batch_size, 1, decoder_seq_len, decoder_seq_len
    ).bool()
    return mask 

#### Протестируем MultiHeadAttention для декодера с маской

In [ ]:
tmp_input = torch.rand(1, 10, 256)
tmp_mask = make_decoder_mask(tmp_input)
print(f"Mask shape: {tmp_mask.shape}")

# Визуализируем
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(tmp_mask[0, 0, :, :])
# Добавляем текстовые надписи
for i in range(tmp_mask.shape[-2]):
    for j in range(tmp_mask.shape[-1]):
        text = plt.text(j, i, tmp_mask[0, 0, i, j].item(), ha="center", va="center", color="red")
plt.show()

In [ ]:
tmp_layer = MultiHeadAttention(
    in_size=10,
    head_size=4,
    num_heads=3,
    out_size=15,
)

tmp_layer

In [ ]:
tmp_input = torch.rand(2, 5, 10)
tmp_mask = make_decoder_mask(tmp_input)

print("Decoder-like input, with mask")
print(f'Input shape: {tmp_input.shape}')
print(f'Mask shape: {tmp_mask.shape}')

tmp_output = tmp_layer(tmp_input, tmp_input, tmp_input, tmp_mask)
print(f'Output shape: {tmp_output.shape}')

del tmp_input, tmp_mask, tmp_output

### 1.2 Positional Encoding

#### Основной класс - PositionalEncoding

**Инициализация:**
* _max_seq_len_ ~ максимальный размер в токенах последовательности
* _emb_size_ ~ размер эмбеддингов на входе

**Forward:**
* _decoder_emb_ ~ эмбеддинги токенов из входа декодера

$$\text{PE}_{(\text{pos}, 2i)} = sin\Bigg( \frac{\text{pos}}{10000^{\frac{2i}{\text{emb_size}}}} \Bigg) \quad ; \quad \text{PE}_{(\text{pos}, 2i + 1)} = cos\Bigg( \frac{\text{pos}}{10000^{\frac{2i}{\text{emb_size}}}} \Bigg)$$

In [ ]:
class PositionalEncoding(nn.Module):
    """
    Class to calculate Positional Encodings, suggested in `Attention is all you need [Vaswaniet al., 2017]`
    """
    def __init__(self, max_seq_len, emb_size):
        """
        Args:
            max_seq_len: max length of input sequence
            emb_size: demension of embedding
        """
        super(PositionalEncoding, self).__init__()
        
        # Запишем все переданые гиперпараметры слоя
        self.max_seq_len = max_seq_len
        self.emb_size = emb_size
        
        # Посчитаем позиционные эмбеддинги в тензорном виде
        ...
        pe = ...
        
        # Добавляем полученный тензор как параметр, который будет сохранятся вместе с моделью, но не будет обучаться
        self.register_buffer('pe', pe)


    def forward(self, decoder_emb):
        """
        Args:
            decoder_emb: decoder sequence after embed
        Returns:
            output: input with positional encodings
        """
        # Тензоры приходят размера batch_size x seq_len x emb_size
        seq_len = decoder_emb.size(1)
        
        # Прибавляем позиционные эмбеддинги
        return ...

#### Протестируем PositionalEncoding

In [ ]:
tmp_layer = PositionalEncoding(
    max_seq_len=5,
    emb_size=10,
)

tmp_layer

In [ ]:
tmp_input = torch.rand(2, 5, 10)

print(f'Input shape: {tmp_input.shape}')
tmp_output = tmp_layer(tmp_input)
print(f'Output shape: {tmp_output.shape}')

del tmp_input, tmp_output

#### Посмотрим на позиционные эмбеддинги

In [ ]:
tmp_layer.pe.shape

Обоснование из статьи [Attention is all you need [Vaswaniet al., 2017]](https://www.semanticscholar.org/reader/204e3073870fae3d05bcbc2f6a8e263d9b72e776):

We chose this function because we hypothesized it would allow the model to easily learn to attend by
relative positions, since for any fixed offset k, $PE_{pos+k}$ can be represented as a linear function of $PE_{pos}$.

In [ ]:
tmp_layer = PositionalEncoding(
    max_seq_len=200,
    emb_size=100,
)

fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(tmp_layer.pe[0, :, :], aspect="auto")
plt.xlabel("emb_size")
plt.ylabel("max_seq_len")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(tmp_layer.pe[0, :, 50:], aspect="auto")
plt.xlabel("emb_size")
plt.ylabel("max_seq_len")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(tmp_layer.pe[0, :, 50:51], aspect="auto")
plt.xlabel("emb_size")
plt.ylabel("max_seq_len")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(tmp_layer.pe[0, 50:51, :], aspect="auto")
plt.xlabel("emb_size")
plt.ylabel("max_seq_len")
plt.show()

### 1.3 Encoder

#### Картинка

<img src="https://www.researchgate.net/profile/Ehsan-Amjadian/publication/352239001/figure/fig1/AS:1033334390013952@1623377525434/Detailed-view-of-a-transformer-encoder-block-It-first-passes-the-input-through-an.jpg" width=500>

#### TransformerEncoderBlock

**Инициализация:**
* _in_size_ ~ размер эмбеддингов на входе
* _head_size_ ~ размер эмбеддингов матриц Q, K, V после преобразования
* _num_heads_ ~ число голов attention
* _out_size_ ~ размер эмбеддингов на выходе attention и блока
* _ff_hidden_size_ ~ размер скрытого представления для линейных слоёв
* _dropout_p_ ~ вероятность для dropout-ов
* _query_in_size_ ~ размер эмбеддингов на входе для query (если None, то in_size)

**Forward:**
* _query, key, value_ ~ 3 тензора (по одному под Q, K и V преобразования - это ещё не сами тензоры $\text{batch_size} \times seq \times d_k$, а тензоры $\text{batch_size} \times seq \times \text{in_size}$)

In [ ]:
class TransformerEncoderBlock(nn.Module):
    """
    Class with one full block within transformer's encoder
    """
    def __init__(self, in_size, head_size, num_heads, out_size, ff_hidden_size, dropout_p=0.2, query_in_size=None):
        """
        Args:
           in_size: input embedding size
           head_size: size of each attention head
           num_heads: number of attention heads
           out_size: output embedding size
           ff_hidden_size: hidden size for feed forward net
           dropout_p: probability for dropout
           query_in_size: embedding size of input for query (if not provided - same as in_size)
        """
        super(TransformerEncoderBlock, self).__init__()
        
        # Запишем все переданые гиперпараметры слоя
        self.in_size = in_size
        self.head_size = head_size
        self.num_heads = num_heads
        self.out_size = out_size
        self.ff_hidden_size = ff_hidden_size
        self.dropout_p = dropout_p
        self.query_in_size = in_size if query_in_size is None else query_in_size
        
        self.attention = ...
        # Если выход и вход attention-а имеют разный размер, то используем линейный слой на residual connection-е
        self.adapt_residual = ...
        
        self.norm_1 = ...
        self.dropout_1 = ...
        
        self.feed_forward = nn.Sequential(OrderedDict([
            ("lin_1", ...),
            ("act", ...),
            ("lin_2", ...),
        ]))

        self.norm_2 = ...
        self.dropout_2 = ...
        

    def forward(self, query, key, value):
        """
        Args:
           block_input: input to corresponding block
        """
        # Получаем на вход 3 тензора batch_size x seq_len x in_size
        attention_out = ...
        attention_residual_out = ...
        norm_1_out = ...

        ff_out = ...
        ff_residual_out = ...
        norm_2_out = ...
        return norm_2_out

#### Протестируем TransformerEncoderBlock для энкодера

In [ ]:
# Проверяем в обычном прямом проходе из энкодера
tmp_layer = TransformerEncoderBlock(
    in_size=10,
    head_size=7,
    num_heads=2,
    out_size=15,
    ff_hidden_size=20,
    dropout_p=0.1,
)

tmp_layer

In [ ]:
tmp_input = torch.rand(2, 5, 10)

print("Encoder-like input")
print(f'Input shape: {tmp_input.shape}')
tmp_output = tmp_layer(tmp_input, tmp_input, tmp_input)
print(f'Output shape: {tmp_output.shape}')

del tmp_input, tmp_output

#### Протестируем TransformerEncoderBlock для декодера

In [ ]:
tmp_layer = TransformerEncoderBlock(
    in_size=10,
    head_size=7,
    num_heads=2,
    out_size=15,
    ff_hidden_size=20,
    dropout_p=0.1,
    query_in_size=12,
)

tmp_layer

In [ ]:
# Проверяем в прямом проходе из декодера, где мы смешиваем информацию из энкодера и декодера
tmp_input_q = torch.rand(2, 5, 12)
tmp_input_kv = torch.rand(2, 7, 10)

print("Encoder+Decoder-like input")
print(f'Input Q shape: {tmp_input_q.shape}')
print(f'Input KV shape: {tmp_input_kv.shape}')

tmp_output = tmp_layer(tmp_input_q, tmp_input_kv, tmp_input_kv)
print(f'Output shape: {tmp_output.shape}')

del tmp_input_q, tmp_input_kv, tmp_output

#### TransformerEncoder

**Инициализация:**
* _max_seq_len_ ~ максимальный размер в токенах последовательности
* _vocab_size_ ~ размер словаря
* _emb_size_ ~ размер эмбеддингов на входе
* _num_layers_ ~ число TransformerEncoderBlock-ов
* _att_out_size_ ~ размер эмбеддингов на выходе attention и блока
* _att_head_size_ ~ размер эмбеддингов матриц Q, K, V после преобразования
* _num_heads_ ~ число голов attention
* _ff_hidden_size_ ~ размер скрытого представления для линейных слоёв
* _dropout_p_ ~ вероятность для dropout-ов

**Forward:**
* _encoder_input_ ~ токены входа в энкодер до эмбеддингов

In [ ]:
class TransformerEncoder(nn.Module):
    """
    Class for encoder within transformer.
    """
    def __init__(self, max_seq_len, vocab_size, emb_size, num_layers, att_out_size, att_head_size, num_heads, ff_hidden_size, dropout_p):
        """
        Args:
            max_seq_len : maximum length of input sequence
            vocab_size: size of the vocabulary
            emb_size: embeddings size
            num_layers: number of encoder layers
            att_out_size: output size for attention and each encoder block
            att_head_size: size of each attention head
            num_heads: number of heads in multihead attention
            ff_hidden_size: hidden size for feed forward net
            dropout_p: probability for dropout
        """
        super(TransformerEncoder, self).__init__()
        
        # Запишем все переданые гиперпараметры слоя
        self.max_seq_len = max_seq_len
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        self.num_layers = num_layers
        self.att_out_size = att_out_size
        self.att_head_size = att_head_size
        self.num_heads = num_heads
        self.ff_hidden_size = ff_hidden_size
        self.dropout_p = dropout_p
        
        self.embedding_layer = nn.Embedding(self.vocab_size, self.emb_size)
        self.positional_encoder = PositionalEncoding(self.max_seq_len, self.emb_size)

        # Красивая обёртка для модулей в dict
        self.encoder_blocks = nn.ModuleDict({
            f"encoder_block_{i}": TransformerEncoderBlock(
                in_size=self.emb_size if i==0 else self.att_out_size,
                head_size=self.att_head_size,
                num_heads=self.num_heads,
                out_size=self.att_out_size,
                ff_hidden_size=self.ff_hidden_size,
                dropout_p=self.dropout_p,
            ) for i in range(self.num_layers)
        })
    
    def forward(self, encoder_input):
        # Получаем на вход batch_size x seq_len
        encoder_emb = self.embedding_layer(encoder_input)  # (batch_size, seq_len, emb_size)
        out = self.positional_encoder(encoder_emb)
        for block in self.encoder_blocks.values():
            out = block(out, out, out)  # (batch_size, seq_len, att_out_size)

        return out

#### Протестируем TransformerEncoder

In [ ]:
tmp_layer = TransformerEncoder(
    max_seq_len=20,
    vocab_size=10000,
    emb_size=10,
    num_layers=2,
    att_head_size=7,
    num_heads=2,
    att_out_size=15,
    ff_hidden_size=20,
    dropout_p=0.1,
)

tmp_layer

In [ ]:
tmp_input = torch.randint(10000, (2, 5))

print(f'Input shape: {tmp_input.shape}')
tmp_output = tmp_layer(tmp_input)
print(f'Output shape: {tmp_output.shape}')

del tmp_input, tmp_output

### 1.4 Decoder

#### Картинка

<img src="https://i.pinimg.com/736x/c1/83/1a/c1831a58ecc935fc0f2ef4d35ce4fddb.jpg" width=500>

#### TransformerDecoderBlock

**Инициализация:**
* _in_size_ ~ размер эмбеддингов на входе
* _head_size_ ~ размер эмбеддингов матриц Q, K, V после преобразования
* _num_heads_ ~ число голов attention
* _out_size_ ~ размер эмбеддингов на выходе attention и блока
* _ff_hidden_size_ ~ размер скрытого представления для линейных слоёв
* _dropout_p_ ~ вероятность для dropout-ов
* _encoder_out_size_ ~ размер эмбеддингов на выходе энкодера (если None, то in_size)

**Forward:**
* _decoder_emb_ ~ тензор, пришедший из предыдущего блока, или эмбеддинги с позиционными
* _encoder_output_ ~ выходной тензор из соответствующего энкодера

In [ ]:
class TransformerDecoderBlock(nn.Module):
    """
    Class with one full block within transformer's decoder
    """
    def __init__(self, in_size, head_size, num_heads, out_size, ff_hidden_size, dropout_p=0.2, encoder_out_size=None):
        """
        Args:
           in_size: input embedding size
           head_size: size of each attention head
           num_heads: number of attention heads
           out_size: output embedding size
           ff_hidden_size: hidden size for feed forward net
           dropout_p: probability for dropout
           encoder_out_size: embedding size of outputs from encoder (if not provided - same as in_size)
        """
        super(TransformerDecoderBlock, self).__init__()
        
        # Запишем все переданые гиперпараметры слоя
        self.in_size = in_size
        self.head_size = head_size
        self.num_heads = num_heads
        self.out_size = out_size
        self.ff_hidden_size = ff_hidden_size
        self.dropout_p = dropout_p
        self.encoder_out_size = in_size if encoder_out_size is None else encoder_out_size
        
        
        self.masked_attention = ...
        # Если выход и вход attention-а имеют разный размер, то используем линейный слой на residual connection-е
        self.adapt_residual = ...
        self.norm = ...
        self.dropout = ...
        self.encoder_block = ...
        
    
    def forward(self, decoder_emb, encoder_output):
        """
        Args:
           decoder_emb: decoder sequence after embed
           encoder_output: output from encoder
        """
        # Получаем на вход тензор batch_size x seq_len x in_size и тензор batch_size x encoder_seq_len x encoder_out_size
        mask = make_decoder_mask(decoder_emb)  # batch_size x 1 x seq_len x seq_len
        attention = ...
        residual_out = ...
        norm_out = ...
        block_out = ...
        
        return block_out

#### Протестируем TransformerDecoderBlock

In [ ]:
tmp_layer = TransformerDecoderBlock(
    in_size=10,
    head_size=7,
    num_heads=2,
    out_size=15,
    ff_hidden_size=20,
    dropout_p=0.1,
    encoder_out_size=12,
)

tmp_layer

In [ ]:
# Проверяем в прямом проходе из декодера, где мы смешиваем информацию из энкодера и декодера
tmp_input_decoder = torch.rand(2, 5, 10)
tmp_output_encoder = torch.rand(2, 7, 12)

print("Encoder+Decoder-like input")
print(f'Decoder input shape: {tmp_input_decoder.shape}')
print(f'Encoder output shape: {tmp_output_encoder.shape}')

tmp_output = tmp_layer(tmp_input_decoder, tmp_output_encoder)
print(f'Output shape: {tmp_output.shape}')

del tmp_input_decoder, tmp_output_encoder, mask

#### TransformerDecoder

**Инициализация:**
* _max_seq_len_ ~ максимальный размер в токенах последовательности
* _vocab_size_ ~ размер словаря
* _emb_size_ ~ размер эмбеддингов на входе
* _num_layers_ ~ число TransformerEncoderBlock-ов
* _att_out_size_ ~ размер эмбеддингов на выходе attention и блока
* _att_head_size_ ~ размер эмбеддингов матриц Q, K, V после преобразования
* _num_heads_ ~ число голов attention
* _ff_hidden_size_ ~ размер скрытого представления для линейных слоёв
* _dropout_p_ ~ вероятность для dropout-ов
* _encoder_out_size_ ~ размер эмбеддингов на выходе энкодера (если None, то in_size)

**Forward:**
* _decoder_input_ ~ токены входа в декодер до эмбеддингов
* _encoder_output_ ~ выходной тензор из соответствующего энкодера

In [ ]:
class TransformerDecoder(nn.Module):
    """
    Class for decoder within transformer.
    """
    def __init__(self, max_seq_len, vocab_size, emb_size, num_layers, att_out_size, att_head_size, num_heads, ff_hidden_size, dropout_p, encoder_out_size=None):
        """  
        Args:
            max_seq_len : maximum length of input sequence
            vocab_size: size of the vocabulary
            emb_size: embeddings size
            num_layers: number of encoder layers
            att_out_size: output size for attention and each encoder block
            att_head_size: size of each attention head
            num_heads: number of heads in multihead attention
            ff_hidden_size: hidden size for feed forward net
            dropout_p: probability for dropout
            encoder_out_size: embedding size of outputs from encoder (if not provided - same as in_size)
        """
        super(TransformerDecoder, self).__init__()
                
        # Запишем все переданые гиперпараметры слоя
        self.max_seq_len = max_seq_len
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        self.num_layers = num_layers
        self.att_out_size = att_out_size
        self.att_head_size = att_head_size
        self.num_heads = num_heads
        self.ff_hidden_size = ff_hidden_size
        self.dropout_p = dropout_p
        self.encoder_out_size = in_size if encoder_out_size is None else encoder_out_size
        
        self.embedding_layer = nn.Embedding(self.vocab_size, self.emb_size)
        self.positional_encoder = ...
        self.dropout = ...

        self.decoder_blocks = nn.ModuleDict({
            f"decoder_block_{i}": ... for i in range(self.num_layers)
        })
        
        self.fc = ...

    def forward(self, decoder_input, encoder_output):
        """
        Args:
            decoder_input:
            encoder_output:
        Returns:
            out: output vector
        """
        # Получаем на вход batch_size x seq_len и batch_size x encoder_seq_len x encoder_out_size
        decoder_emb = self.embedding_layer(decoder_input)  # batch_size x seq_len x emb_size
        decoder_emb_pos = ...
        
        out = ...
     
        for block in self.decoder_blocks.values():
            out = ...

        logits = ...
        
        return logits

#### Протестируем TransformerDecoder

In [ ]:
tmp_layer = TransformerDecoder(
    max_seq_len=20,
    vocab_size=10000,
    emb_size=10,
    num_layers=2,
    att_head_size=7,
    num_heads=2,
    att_out_size=15,
    ff_hidden_size=20,
    dropout_p=0.1,
    encoder_out_size=12,
)

tmp_layer

In [ ]:
# Проверяем в прямом проходе из декодера, где мы смешиваем информацию из энкодера и декодера
tmp_input_decoder = torch.randint(10000, (2, 5))
tmp_output_encoder = torch.rand(2, 7, 12)

print("Encoder+Decoder-like input")
print(f'Decoder input shape: {tmp_input_decoder.shape}')
print(f'Encoder output shape: {tmp_output_encoder.shape}')

tmp_output = tmp_layer(tmp_input_decoder, tmp_output_encoder)
print(f'Output shape: {tmp_output.shape}')

del tmp_input_decoder, tmp_output_encoder

### 1.5 Transformer

In [ ]:
class Transformer(nn.Module):
    """
    Class for full encoder-deccoder transformer
    """
    def __init__(
        self,
        max_seq_len,
        vocab_size,
        emb_size,
        
        num_encoder_layers,
        enc_att_out_size,
        enc_att_head_size,
        enc_num_heads,
        enc_ff_hidden_size,
        enc_dropout_p,
        
        num_decoder_layers,
        dec_att_out_size,
        dec_att_head_size,
        dec_num_heads,
        dec_ff_hidden_size,
        dec_dropout_p,
    ):
        super(Transformer, self).__init__()
        
        # Запишем все переданые гиперпараметры модели
        self.max_seq_len = max_seq_len
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        
        self.num_encoder_layers = num_encoder_layers
        self.enc_att_out_size = enc_att_out_size
        self.enc_att_head_size = enc_att_head_size
        self.enc_num_heads = enc_num_heads
        self.enc_ff_hidden_size = enc_ff_hidden_size
        self.enc_dropout_p = enc_dropout_p
        
        self.num_decoder_layers = num_decoder_layers
        self.dec_att_out_size = dec_att_out_size
        self.dec_att_head_size = dec_att_out_size
        self.dec_num_heads = dec_num_heads
        self.dec_ff_hidden_size = dec_ff_hidden_size
        self.dec_dropout_p = dec_dropout_p

        # Encoder
        self.encoder = TransformerEncoder(
            max_seq_len=self.max_seq_len,
            vocab_size=self.vocab_size,
            emb_size=self.emb_size,
            num_layers=self.num_encoder_layers,
            att_head_size=self.enc_att_head_size,
            num_heads=self.enc_num_heads,
            att_out_size=self.enc_att_out_size,
            ff_hidden_size=self.enc_ff_hidden_size,
            dropout_p=self.enc_dropout_p,
        )
        
        # Decoder
        self.decoder = TransformerDecoder(
            max_seq_len=self.max_seq_len,
            vocab_size=self.vocab_size,
            emb_size=self.emb_size,
            num_layers=self.num_decoder_layers,
            att_head_size=self.dec_att_head_size,
            num_heads=self.dec_num_heads,
            att_out_size=self.dec_att_out_size,
            ff_hidden_size=self.dec_ff_hidden_size,
            dropout_p=self.dec_dropout_p,
            encoder_out_size=self.enc_att_out_size,
        )
    
    def forward(self, encoder_input, decoder_input):
        """
        Args:
            encoder_input: input to encoder 
            decoder_input: input to decoder
        out:
            out: final tensor with logits of each word in vocab
        """
        # Получаем на вход batch_size x enc_seq_len и batch_size x dec_seq_len
        encoder_output = self.encoder(encoder_input)  # (batch_size, enc_seq_len, enc_att_out_size)
   
        return self.decoder(decoder_input, encoder_output)  # (batch_size, dec_seq_len, vocab_size)

### 1.6 Тестируем

In [ ]:
tmp_layer = Transformer(
    max_seq_len=20,
    vocab_size=10000,
    emb_size=10,
    
    num_encoder_layers=3,
    enc_att_head_size=7,
    enc_num_heads=3,
    enc_att_out_size=20,
    enc_ff_hidden_size=30,
    enc_dropout_p=0.2,
    
    num_decoder_layers=2,
    dec_att_head_size=7,
    dec_num_heads=2,
    dec_att_out_size=15,
    dec_ff_hidden_size=20,
    dec_dropout_p=0.1,
)

tmp_layer

In [ ]:
tmp_input_encoder = torch.randint(10000, (2, 9))
tmp_input_decoder = torch.randint(10000, (2, 5))

print(f'Encoder input shape: {tmp_input_encoder.shape}')
print(f'Decoder input shape: {tmp_input_decoder.shape}')

tmp_output = tmp_layer(tmp_input_encoder, tmp_input_decoder)
print(f'Output shape: {tmp_output.shape}')

del tmp_input_decoder, tmp_input_encoder

### 1.7 Дополнительно

In [ ]:
?torch.nn.Transformer